In [8]:
import os
import sys
import argparse
import pandas as pd

In [9]:
with open('/home/jugalde/germlab/genome_test/samples.txt') as f:
    sample_ID = f.read().splitlines()

raw_folder = '/home/jugalde/germlab/genome_test/consolidated//'
batch = 'mrsa_genomes'

len(sample_ID)

275

In [10]:
## Info about the number of reads 

fastp = pd.read_table(raw_folder+'fastp_results.tsv',delimiter = '\t')
fastp_clean = pd.DataFrame().assign(sample_ID = fastp["sample"], n_reads_before_trimming = fastp["reads_before"],
                                   n_read_after_trimming = fastp["reads_after"])

## Info about the data obtained from BRAKEN

# braken = pd.read_table(raw_folder+'bracken_results.tsv',delimiter = '\t')
# braken_clean = pd.DataFrame(columns=['sample_ID', 'braken_species_first', 'braken_taxonomy_id_first',
#                                      'braken_fraction_reads_first', 'braken_species_second',
#                                      'braken_taxonomy_id_second', 'braken_fraction_reads_second'])

# ## Selects the two highest hit from bracken and organizes the info
# for i in range(len(sample_ID)):
#     braken_temp = braken[braken["sample"] == sample_ID[i]].sort_values(by='fraction_total_reads', ascending = False)
    
#     if braken_temp.shape[0] > 0:
#         braken_species_first = braken_temp.iloc[0]['name']
#         braken_taxonomy_id_first = braken_temp.iloc[0]['taxonomy_id']
#         braken_fraction_reads_first = braken_temp.iloc[0]['fraction_total_reads']
#     else:
#         braken_species_first = None
#         braken_taxonomy_id_first = None
#         braken_fraction_reads_first = None

#     if braken_temp.shape[0] > 1:
#         braken_species_second = braken_temp.iloc[1]['name']
#         braken_taxonomy_id_second = braken_temp.iloc[1]['taxonomy_id']
#         braken_fraction_reads_second = braken_temp.iloc[1]['fraction_total_reads']
#     else:
#         braken_species_second = None
#         braken_taxonomy_id_second = None
#         braken_fraction_reads_second = None
     
#     braken_temp_row = [sample_ID[i],braken_species_first,braken_taxonomy_id_first,braken_fraction_reads_first,
#                   braken_species_second,braken_taxonomy_id_second,braken_fraction_reads_second]

#     braken_clean.loc[len(braken_clean)] = braken_temp_row

## Info about the assembly quality

qc_assembly = pd.read_table(raw_folder+'qc_assembly.tsv',delimiter = '\t')
qc_assembly_clean = pd.DataFrame().assign(sample_ID = qc_assembly["sample"],
#                                          contigs_0 = qc_assembly["contigs_0"],
#                                          contigs_10k = qc_assembly["contigs_10k"],
#                                          contigs_50k = qc_assembly["contigs_50k"],
                                          length_total_0 = qc_assembly["contigs_total"],
                                          contigs = qc_assembly["length_50k"],
                                          length_total = qc_assembly["largest_contig"],
                                          largest_contig = qc_assembly["length_total"],
                                          GC_content = qc_assembly["gc"],
                                          N50 = qc_assembly["n50"])

## Info about the MLST results

mlst = pd.read_table(raw_folder+'mlst_results.tsv',delimiter = '\t')
mlst_clean = mlst.rename(columns = {'sample':'sample_ID','species':"MLST_species"})

## Info about the AMR finder results

amrfinder = pd.read_table(raw_folder+'amrfinder_results.tsv',delimiter = '\t')

amrfinder_carb_clean = pd.DataFrame(columns=['sample_ID', 'carbapenemases', 'IMP', 'KPC', 'NDM', 'VIM',
                                             'others_carbapenem', 'point_mut_carbapenem'])

## Organizes the AMR info starting by Carbapenemases

for i in range(len(sample_ID)):
    amrfinder_temp = amrfinder[amrfinder["sample"] == sample_ID[i]]

    amrfinder_temp_carb = amrfinder_temp[amrfinder_temp["Subclass"] == "CARBAPENEM"]
    amrfinder_temp_carb_amr = amrfinder_temp_carb[amrfinder_temp_carb["Element subtype"] == "AMR"]
    amrfinder_temp_carb_point = amrfinder_temp_carb[amrfinder_temp_carb["Element subtype"] == "POINT"]

    amrfinder_temp_carb_IMP_table = amrfinder_temp_carb_amr[amrfinder_temp_carb_amr['Gene symbol'].str.match("blaIMP")]
    amrfinder_temp_carb_KPC_table = amrfinder_temp_carb_amr[amrfinder_temp_carb_amr['Gene symbol'].str.match("blaKPC")]
    amrfinder_temp_carb_NDM_table = amrfinder_temp_carb_amr[amrfinder_temp_carb_amr['Gene symbol'].str.match("blaNDM")]
    amrfinder_temp_carb_VIM_table = amrfinder_temp_carb_amr[amrfinder_temp_carb_amr['Gene symbol'].str.match("blaVIM")]
    amrfinder_temp_carb_others_table = amrfinder_temp_carb_amr[~amrfinder_temp_carb_amr['Gene symbol'].
                                                               str.match("blaKPC|blaNDM|blaIMP|blaVIM")]
    if amrfinder_temp_carb_amr.shape[0] > 0:
        carbapenemases_temp = "Yes"
    else:
        carbapenemases_temp = "No"

    if amrfinder_temp_carb_IMP_table.shape[0] > 0:
        IMP_temp = [i.rsplit(" ",1)[-1] for i in amrfinder_temp_carb_IMP_table["Name of closest sequence"].to_list()]
        IMP_temp = ", ".join(IMP_temp)
    else:
        IMP_temp = None

    if amrfinder_temp_carb_KPC_table.shape[0] > 0:
        KPC_temp = [i.rsplit(" ",1)[-1] for i in amrfinder_temp_carb_KPC_table["Name of closest sequence"].to_list()]
        KPC_temp = ", ".join(KPC_temp)
    else:
        KPC_temp = None
        
    if amrfinder_temp_carb_NDM_table.shape[0] > 0:
        NDM_temp = [i.rsplit(" ",1)[-1] for i in amrfinder_temp_carb_NDM_table["Name of closest sequence"].to_list()]
        NDM_temp = ", ".join(NDM_temp)
    else:
        NDM_temp = None

    if amrfinder_temp_carb_VIM_table.shape[0] > 0:
        VIM_temp = [i.rsplit(" ",1)[-1] for i in amrfinder_temp_carb_VIM_table["Name of closest sequence"].to_list()]
        VIM_temp = ", ".join(VIM_temp)
    else:
        VIM_temp = None

    if amrfinder_temp_carb_others_table.shape[0] > 0:
        others_temp = [i.rsplit(" ",1)[-1] for i in amrfinder_temp_carb_others_table["Name of closest sequence"].to_list()]
        others_temp = ", ".join(others_temp)
    else:
        others_temp = None

    if amrfinder_temp_carb_point.shape[0] > 0:
        point_temp = ", ".join(amrfinder_temp_carb_point["Gene symbol"].to_list())
    else:
        point_temp = None

    amrfinder_temp_row = [sample_ID[i], carbapenemases_temp, IMP_temp, KPC_temp, NDM_temp, VIM_temp, others_temp,
                          point_temp]

    amrfinder_carb_clean.loc[len(amrfinder_carb_clean)] = amrfinder_temp_row

## Organizes the AMR info for ESBLs and ESBLs of interest

amrfinder_esbl_clean = pd.DataFrame(columns=['sample_ID', 'ESBL', 'CTX-M', 'SHV', 'ESBL_TEM', 'others_ESBL',
                                       'point_mut_beta_lactam', 'all_TEM'])

for i in range(len(sample_ID)):
    amrfinder_temp = amrfinder[amrfinder["sample"] == sample_ID[i]]
    amrfinder_temp_esbl = amrfinder_temp[amrfinder_temp["Name of closest sequence"].
                                         str.contains("extended-spectrum|broad-spectrum")]
    amrfinder_temp_esbl = amrfinder_temp_esbl[amrfinder_temp_esbl["Name of closest sequence"].
                                              str.contains("beta-lactamase")]
    amrfinder_temp_esbl_amr = amrfinder_temp_esbl[amrfinder_temp_esbl["Element subtype"] == "AMR"]
    amrfinder_temp_esbl_point = amrfinder_temp_esbl[amrfinder_temp_esbl["Element subtype"] == "POINT"]

    amrfinder_temp_esbl_CTX_M_table = amrfinder_temp_esbl_amr[amrfinder_temp_esbl_amr['Gene symbol'].str.match("blaCTX-M")]
    amrfinder_temp_esbl_SHV_table = amrfinder_temp_esbl_amr[amrfinder_temp_esbl_amr['Gene symbol'].str.match("blaSHV")]
    amrfinder_temp_esbl_TEM_table = amrfinder_temp_esbl_amr[amrfinder_temp_esbl_amr['Gene symbol'].str.match("blaTEM")]
    amrfinder_temp_esbl_others_table = amrfinder_temp_esbl_amr[~amrfinder_temp_esbl_amr['Gene symbol'].
                                                               str.match("blaCTX-M|blaSHV|blaTEM")]
    
    amrfinder_temp_all_TEM_table = amrfinder_temp[amrfinder_temp['Gene symbol'].str.match("blaTEM")]

    if amrfinder_temp_esbl_amr.shape[0] > 0:
        ESBL_temp = "Yes"
    else:
        ESBL_temp = "No"

    if amrfinder_temp_esbl_CTX_M_table.shape[0] > 0:
        CTX_M_temp = [i.rsplit(" ",1)[-1] for i in amrfinder_temp_esbl_CTX_M_table["Name of closest sequence"].to_list()]
        CTX_M_temp = ", ".join(CTX_M_temp)
    else:
        CTX_M_temp = None
        
    if amrfinder_temp_esbl_SHV_table.shape[0] > 0:
        SHV_temp = [i.rsplit(" ",1)[-1] for i in amrfinder_temp_esbl_SHV_table["Name of closest sequence"].to_list()]
        SHV_temp = ", ".join(SHV_temp)
    else:
        SHV_temp = None
        
    if amrfinder_temp_esbl_TEM_table.shape[0] > 0:
        ESBL_TEM_temp = [i.rsplit(" ",1)[-1] for i in amrfinder_temp_esbl_TEM_table["Name of closest sequence"].to_list()]
        ESBL_TEM_temp = ", ".join(ESBL_TEM_temp)
    else:
        ESBL_TEM_temp = None
        
    if amrfinder_temp_esbl_others_table.shape[0] > 0:
        others_temp = [i.rsplit(" ",1)[-1] for i in amrfinder_temp_esbl_others_table["Name of closest sequence"].to_list()]
        others_temp = ", ".join(others_temp)
    else:
        others_temp = None
    
    if amrfinder_temp_esbl_point.shape[0] > 0:
        point_temp = ", ".join(amrfinder_temp_esbl_point["Gene symbol"].to_list())
    else:
        point_temp = None
    
    if amrfinder_temp_all_TEM_table.shape[0] > 0:
        all_TEM_temp = [i.rsplit(" ",1)[-1] for i in amrfinder_temp_all_TEM_table["Name of closest sequence"].to_list()]
        all_TEM_temp = ", ".join(all_TEM_temp)
    else:
        all_TEM_temp = None

    amrfinder_temp_row = [sample_ID[i], ESBL_temp, CTX_M_temp, SHV_temp, ESBL_TEM_temp, others_temp, point_temp, all_TEM_temp]

    amrfinder_esbl_clean.loc[len(amrfinder_esbl_clean)] = amrfinder_temp_row

## Organizes the AMR info for mec and van

amrfinder_mec_van_clean = pd.DataFrame(columns=['sample_ID', 'mecA', 'others_mec', 'VanA', 'VanB', 'others_van'])

for i in range(len(sample_ID)):
    amrfinder_temp = amrfinder[amrfinder["sample"] == sample_ID[i]]

    amrfinder_temp_mec_table = amrfinder_temp[amrfinder_temp['Gene symbol'].str.match("mec")]
    amrfinder_temp_meca_table = amrfinder_temp_mec_table[amrfinder_temp_mec_table['Gene symbol'].str.match("mecA")]
    amrfinder_temp_mec_others_table = amrfinder_temp_mec_table[~amrfinder_temp_mec_table['Gene symbol'].
                                                               str.match("mecA")]
    
    amrfinder_temp_van_table = amrfinder_temp[amrfinder_temp['Gene symbol'].str.match("van")]
    amrfinder_temp_vana_table = amrfinder_temp_van_table[amrfinder_temp_van_table['Gene symbol'].str.match("^vanA")]
    amrfinder_temp_vanb_table = amrfinder_temp_van_table[amrfinder_temp_van_table['Gene symbol'].str.match("^vanB")]
    amrfinder_temp_van_others_table = amrfinder_temp_van_table[~amrfinder_temp_van_table['Gene symbol'].
                                                               str.match("vanA|vanB")]
    if amrfinder_temp_meca_table.shape[0] > 0:
        meca_temp = ", ".join(amrfinder_temp_meca_table["Gene symbol"].to_list())
    else:
        meca_temp = None

    if amrfinder_temp_mec_others_table.shape[0] > 0:
        mec_others_temp = ", ".join(amrfinder_temp_mec_others_table["Gene symbol"].to_list())
    else:
        mec_others_temp = None

    if amrfinder_temp_vana_table.shape[0] > 0:
        vana_temp = ", ".join(amrfinder_temp_vana_table["Gene symbol"].to_list())
    else:
        vana_temp = None
        
    if amrfinder_temp_vanb_table.shape[0] > 0:
        vanb_temp = ", ".join(amrfinder_temp_vanb_table["Gene symbol"].to_list())
    else:
        vanb_temp = None

    if amrfinder_temp_van_others_table.shape[0] > 0:
        van_others_temp = ", ".join(amrfinder_temp_van_others_table["Gene symbol"].to_list())
    else:
        van_others_temp = None
        
    amrfinder_temp_row = [sample_ID[i], meca_temp, mec_others_temp, vana_temp, vanb_temp, van_others_temp]

    amrfinder_mec_van_clean.loc[len(amrfinder_mec_van_clean)] = amrfinder_temp_row
    
## Info about the SCCmec

if os.path.isfile(raw_folder+'sccmec.tsv'):
    sccmec = pd.read_table(raw_folder+'sccmec.tsv',delimiter = '\t')    
else:
    sccmec = pd.DataFrame(columns=['sample', 'sccmec'])
    
sccmec_clean = pd.DataFrame(columns=['sample_ID', 'sccmec'])

for i in range(len(sample_ID)):
    sccmec_temp = sccmec[sccmec["sample"] == sample_ID[i]]
    if sccmec_temp.shape[0] == 1:
        sccmec_data = sccmec_temp["sccmec"].iloc[0]
    else:
        sccmec_data = "NA"
    
    sccmec_temp_row = [sample_ID[i], sccmec_data]
    
    sccmec_clean.loc[len(sccmec_clean)] =  sccmec_temp_row

In [11]:
summary_table = pd.DataFrame({'batch':([batch]*len(sample_ID)),'sample_ID':sample_ID})
summary_table = pd.merge(summary_table,fastp_clean,how='left', on='sample_ID')
#summary_table = pd.merge(summary_table,braken_clean,how='left', on='sample_ID')
summary_table = pd.merge(summary_table,qc_assembly_clean,how='left', on='sample_ID')
summary_table = pd.merge(summary_table,mlst_clean,how='left', on='sample_ID')
summary_table = pd.merge(summary_table,amrfinder_carb_clean,how='left', on='sample_ID')
summary_table = pd.merge(summary_table,amrfinder_esbl_clean,how='left', on='sample_ID')
summary_table = pd.merge(summary_table,amrfinder_mec_van_clean,how='left', on='sample_ID')
summary_table = pd.merge(summary_table,sccmec_clean,how='left', on='sample_ID')
summary_table.shape[0]

275

In [12]:
summary_table.to_csv("/home/jugalde/germlab/genome_test/20250514_Summary.csv",sep="\t")